In [30]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

In [ ]:
# Set based on whether running locally or on colab
COLAB = True

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    #torch.backends.cudnn.enabled = False

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
if COLAB:
  ! unzip "/content/gdrive/MyDrive/Colab Notebooks/data/dataset.zip"

Archive:  /content/gdrive/MyDrive/Colab Notebooks/data/dataset.zip
replace __MACOSX/._dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [46]:
path_data = './MnM2/dataset/'
path_out = './data/mnm/'

In [56]:
dir_list = [os.path.join(path_out, 'train/images'), os.path.join(path_out, 'train/labels'), os.path.join(path_out, 'valid/images'), os.path.join(path_out, 'valid/labels'), os.path.join(path_out, 'test/images'), os.path.join(path_out, 'test/labels') ]

for dir in dir_list:
    os.makedirs(dir, exist_ok=True)

In [48]:
def resize_nii(original, mask=False):
    if mask:
        order = 0
    else:
        order = 3
        
    new_shape = (256, 256, original.shape[2]) 
    zoom_factors = [n / float(o) for n, o in zip(new_shape, original.shape)]
    resized_data = zoom(original, zoom_factors, order=order) # nearest neightbour interpolation ensures output only has discete values
    return resized_data

In [57]:
for (dirpath, dirnames, filenames) in os.walk(path_data):
    for file in sorted(filenames, reverse=True):
      if file == '.DS_Store':
          continue
      if (1 <= int(file.split('_')[0]) <= 100) or (161 <= int(file.split('_')[0]) <= 175) or (201 <= int(file.split('_')[0]) <= 280):
        if int(file.split('_')[0]) <= 160:
          path_out_ = path_out+'train/'
        elif (160 < int(file.split('_')[0]) <= 200):
          path_out_ = path_out+'valid/'
        else:
          path_out_ = path_out+'test/'

        if file.split('_')[1] == 'SA':
            dir_patient = file.split('_')[0]
            if file.split('_')[2].split('.')[0] != 'CINE':
                # print(os.path.join(path_data+dir_patient+'/', file))
                if 'gt' not in file:
                    original = nib.load(os.path.join(path_data+dir_patient+'/', file)).get_fdata()
                    affine = nib.load(os.path.join(path_data+dir_patient+'/', file)).affine
                    hdr = nib.load(os.path.join(path_data+dir_patient+'/', file)).header
                    
                    # resize image
                    img = resize_nii(original)

                    for i in range(original.shape[2]):
                        #out = nib.Nifti1Image(original, affine, header=hdr)
                        out = nib.Nifti1Image(img[:, :, i], affine, header=hdr)                      
                        nib.save(out, os.path.join(path_out_+'images/', file.split('.')[0]+'_'+format(i,"04")+'.nii.gz'))
                else:
                    original_gt = nib.load(os.path.join(path_data+dir_patient+'/', file.split('.')[0]+'.nii.gz')).get_fdata()
                    affine_gt = nib.load(os.path.join(path_data+dir_patient+'/', file.split('.')[0]+'.nii.gz')).affine
                    hdr_gt = nib.load(os.path.join(path_data+dir_patient+'/', file.split('.')[0]+'.nii.gz')).header
                    
                    # resize image
                    img_gt = resize_nii(original_gt, mask=True)
                    
                    for i in range(original_gt.shape[2]):
                        #out_gt = nib.Nifti1Image(original_gt, affine_gt, header=hdr_gt)
                        out_gt = nib.Nifti1Image(img_gt[:, :, i], affine_gt, header=hdr_gt)
                        nib.save(out_gt, os.path.join(path_out_+'labels/', file.split('.')[0]+'_'+format(i,"04")+'.nii.gz'))

168 168_SA_ES_gt.nii.gz
168 168_SA_ES.nii.gz
168 168_SA_ED_gt.nii.gz
168 168_SA_ED.nii.gz
166 166_SA_ES_gt.nii.gz
166 166_SA_ES.nii.gz
166 166_SA_ED_gt.nii.gz
166 166_SA_ED.nii.gz
161 161_SA_ES_gt.nii.gz
161 161_SA_ES.nii.gz
161 161_SA_ED_gt.nii.gz
161 161_SA_ED.nii.gz
167 167_SA_ES_gt.nii.gz
167 167_SA_ES.nii.gz
167 167_SA_ED_gt.nii.gz
167 167_SA_ED.nii.gz
169 169_SA_ES_gt.nii.gz
169 169_SA_ES.nii.gz
169 169_SA_ED_gt.nii.gz
169 169_SA_ED.nii.gz
024 024_SA_ES_gt.nii.gz
024 024_SA_ES.nii.gz
024 024_SA_ED_gt.nii.gz
024 024_SA_ED.nii.gz
216 216_SA_ES_gt.nii.gz
216 216_SA_ES.nii.gz
216 216_SA_ED_gt.nii.gz
216 216_SA_ED.nii.gz
229 229_SA_ES_gt.nii.gz
229 229_SA_ES.nii.gz
229 229_SA_ED_gt.nii.gz
229 229_SA_ED.nii.gz
211 211_SA_ES_gt.nii.gz
211 211_SA_ES.nii.gz
211 211_SA_ED_gt.nii.gz
211 211_SA_ED.nii.gz
023 023_SA_ES_gt.nii.gz
023 023_SA_ES.nii.gz
023 023_SA_ED_gt.nii.gz
023 023_SA_ED.nii.gz
227 227_SA_ES_gt.nii.gz
227 227_SA_ES.nii.gz
227 227_SA_ED_gt.nii.gz
227 227_SA_ED.nii.gz
015 015_SA

In [58]:
# iterate over all files and remove those that do not have a RV mask
for (dirpath, dirnames, filenames) in os.walk(path_out):
    for file in sorted(filenames, reverse=True):
        if 'gt' in file: 
            mask_path = dirpath.split('/')
            del mask_path[-1]
            img_path = '/'.join(mask_path) + '/images'
            
            
            img_file = file.split('_')
            del img_file[3]
            img_filename = '_'.join(img_file)
            original_gt = nib.load(os.path.join(dirpath, file)).get_fdata()
            
            if not len(original_gt[original_gt==3.0]) > 0:
                # delete mask and image
                
                print(f'Removing Image:{os.path.join(img_path, img_filename)}')
                print(f'Removing Mask:{os.path.join(dirpath, file)}')
                os.remove(os.path.join(dirpath, file))
                os.remove(os.path.join(img_path, img_filename))
            
            

Removing Image:./data/mnm/valid/images/175_SA_ES_0011.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0011.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0010.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0010.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0009.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0009.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0008.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0008.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0007.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0007.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0001.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0001.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ES_0000.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ES_gt_0000.nii.gz
Removing Image:./data/mnm/valid/images/175_SA_ED_0011.nii.gz
Removing Mask:./data/mnm/valid/labels/175_SA_ED_gt_0011.nii.gz
Removing

In [25]:
if COLAB:
  !zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data_slices.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data_slices"

NameError: name 'COLAB' is not defined

In [ ]:
#if COLAB:
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_train_images.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/images"
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_train_labels.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels"
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_valid_images.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/valid/images"
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_valid_labels.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/valid/labels"
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_test_images.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/test/images"
  #!zip -r "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/mnm2_test_labels.zip" "/content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/test/labels"

Streaming output truncated to the last 5000 lines.
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ES_gt_0010.nii.gz (deflated 80%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0000.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0001.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0002.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0003.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0004.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0005.nii.gz (deflated 74%)
  adding: content/gdrive/MyDrive/Colab Notebooks/data/mnm2_data/train/labels/019_SA_ED_gt_0006.nii.gz (deflated 74%)
  adding: con